In [169]:
import pandas as pd

location = pd.read_csv(r'C:\Users\imran\Desktop\MyData\Python\Project\city.csv')
uses = pd.read_csv(r'C:\Users\imran\Desktop\MyData\Python\Project\consumption.csv')
tariff = pd.read_csv(r'C:\Users\imran\Desktop\MyData\Python\Project\tariff.csv')
df_uses = pd.read_csv(r'C:\Users\imran\Desktop\MyData\Python\Project\modified.csv')

tariff

,city,month,tariff_per_unit
0,Mumbai,2022-01,6.79
1,Mumbai,2022-02,6.94
2,Mumbai,2022-03,5.53
3,Mumbai,2022-04,5.80
4,Mumbai,2022-05,7.05
...,...,...,...
67,Pune,2022-08,4.76
68,Pune,2022-09,6.83
69,Pune,2022-10,7.04
70,Pune,2022-11,5.05


In [42]:
location['city'] = location['city'].str.title()

In [47]:
print(f"{10000000:,}")

10,000,000


In [57]:
def con(x):
    y =  f"{int(x):,}"
    return y
print(con('1000000'))

1,000,000


In [170]:
location['population'] = location['population'].str.replace(',','')
location['population'] = location['population'].apply(lambda x: f"{int(x):,}")
location['population']

0    12,442,373
1    11,034,555
2     8,436,675
3     7,088,000
4     4,486,679
5     3,124,458
Name: population, dtype: object

In [242]:
location['population'] = location['population'].str.replace(',','')
location['population'] = location['population'].apply(con)
location['population']

location.to_csv(r'C:\Users\imran\Desktop\MyData\Python\Project\city.csv', index = False)

NameError: name 'con' is not defined

In [82]:
print(location)

        city region  population
0     Mumbai   West  12,442,373
1      Delhi  North  11,034,555
2  Bangalore  South   8,436,675
3    Chennai  South   7,088,000
4    Kolkata   East   4,486,679
5       Pune   West   3,124,458


In [83]:
uses

,city,date,kwh_consumed,month
0,Mumbai,2022-01-01,5596.06,2022-01
1,Mumbai,2022-01-02,4834.08,2022-01
2,Mumbai,2022-01-03,4719.02,2022-01
3,Mumbai,2022-01-04,4719.04,2022-01
4,Mumbai,2022-01-05,6895.06,2022-01
...,...,...,...,...
2185,Pune,2022-12-27,4764.46,2022-12
2186,Pune,2022-12-28,2787.45,2022-12
2187,Pune,2022-12-29,3931.62,2022-12
2188,Pune,2022-12-30,2364.65,2022-12


In [94]:
uses['date'] = pd.to_datetime(uses['date'], format = '%Y,%m,%d')

uses['date'].dt.day_name()

0        Saturday
1          Sunday
2          Monday
3         Tuesday
4       Wednesday
          ...    
2185      Tuesday
2186    Wednesday
2187     Thursday
2188       Friday
2189     Saturday
Name: date, Length: 2190, dtype: object

In [157]:
uses_m = uses.merge(location,left_on = 'city', right_on='city', how = 'left')
uses_m['date'] = pd.to_datetime (uses_m['date'], format = '%Y-%m-%d')
uses_m['day'] = uses_m['date'].dt.day_name()
uses_m['month'] = uses_m['date'].dt.month_name()
uses_m['year'] = uses_m['date'].dt.year
uses_m['weekday'] = uses_m['date'].dt.weekday

uses_m['monyr'] = uses_m['date'].dt.strftime ('%Y-%m')
uses_m = uses_m.merge(tariff, left_on = ['city','monyr'], right_on=['city','month'], how = 'left')
uses_m= uses_m.drop(columns =['month_y',])
df_uses =uses_m
df_uses.to_csv(r'C:\Users\imran\Desktop\MyData\Python\Project\modified.csv')

In [49]:

df_uses['bill_amt'] = (df_uses['kwh_consumed']*df_uses['tariff_per_unit']).round(2)

df_uses.pivot_table(index= 'month_x', columns = 'region', values = ['kwh_consumed','bill_amt'],
                    aggfunc = {'kwh_consumed':'sum','bill_amt': 'mean'}, margins = True, margins_name= 'total')


df_uses

,Unnamed: 0,city,date,kwh_consumed,month_x,region,population,day,year,weekday,monyr,tariff_per_unit,bill_amt
0,0,Mumbai,2022-01-01,5596.06,January,West,"12,442,373",Saturday,2022,5,2022-01,6.79,37997.25
1,1,Mumbai,2022-01-02,4834.08,January,West,"12,442,373",Sunday,2022,6,2022-01,6.79,32823.40
2,2,Mumbai,2022-01-03,4719.02,January,West,"12,442,373",Monday,2022,0,2022-01,6.79,32042.15
3,3,Mumbai,2022-01-04,4719.04,January,West,"12,442,373",Tuesday,2022,1,2022-01,6.79,32042.28
4,4,Mumbai,2022-01-05,6895.06,January,West,"12,442,373",Wednesday,2022,2,2022-01,6.79,46817.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,2185,Pune,2022-12-27,4764.46,December,West,"3,124,458",Tuesday,2022,1,2022-12,5.79,27586.22
2186,2186,Pune,2022-12-28,2787.45,December,West,"3,124,458",Wednesday,2022,2,2022-12,5.79,16139.34
2187,2187,Pune,2022-12-29,3931.62,December,West,"3,124,458",Thursday,2022,3,2022-12,5.79,22764.08
2188,2188,Pune,2022-12-30,2364.65,December,West,"3,124,458",Friday,2022,4,2022-12,5.79,13691.32


In [38]:
df_uses.groupby(['city','month_x'])['kwh_consumed'].max()

city       month_x  
Bangalore  April        8122.02
           August       8107.48
           December     7488.90
           February     6788.87
           January      7991.60
                         ...   
Pune       March        7725.74
           May          7463.57
           November     6563.93
           October      7450.49
           September    8891.71
Name: kwh_consumed, Length: 72, dtype: float64

In [86]:
t_consumption = df_uses.pivot_table(index= 'city',values = 'kwh_consumed', aggfunc = ('sum'))

per_capita_consumption = t_consumption.merge(location, left_on = 'city', right_on = 'city', how = 'left').drop(columns = 'region')

per_capita_consumption['per_capita_consumption']=per_capita_consumption['kwh_consumed']/(per_capita_consumption['population'].replace (',','',regex= True).astype(int))*100

per_capita_consumption


#per_capita_consumption['kwh_consumed'] = (
 #   per_capita_consumption['kwh_consumed']/
  #  per_capita_consumption['population'].replace (',','',regex= True).astype(int))*100
#per_capita_consumption

#per_capita_consumption['kwh_consumed']/(per_capita_consumption['population'].replace (',','',regex= True).astype(int))*100





,city,kwh_consumed,population,per_capita_consumption
0,Bangalore,1827043.72,"8,436,675",21.655969
1,Chennai,1789114.52,"7,088,000",25.241458
2,Delhi,1848933.67,"11,034,555",16.755852
3,Kolkata,1800848.03,"4,486,679",40.137662
4,Mumbai,1820412.66,"12,442,373",14.630751
5,Pune,1818824.92,"3,124,458",58.212494


In [95]:
#df_uses.groupby(['city','population','kwh_consumed']).agg({'population':'sum','kwh_consumed':'sum'})

df_per_capita_uses= df_uses.pivot_table(index= ['city','population'],values = ['kwh_consumed'], aggfunc=({'kwh_consumed':'sum'})). reset_index()

df_per_capita_uses['per_capita'] = df_per_capita_uses ['kwh_consumed']/(df_per_capita_uses['population'].replace(',','', regex = True).astype(int))*100

tariff.pivot_table(index = 'city', values = 'tariff_per_unit', aggfunc = ('sum'))
df_uses

,Unnamed: 0,city,date,kwh_consumed,month_x,region,population,day,year,weekday,monyr,tariff_per_unit
0,0,Mumbai,2022-01-01,5596.06,January,West,"12,442,373",Saturday,2022,5,2022-01,6.79
1,1,Mumbai,2022-01-02,4834.08,January,West,"12,442,373",Sunday,2022,6,2022-01,6.79
2,2,Mumbai,2022-01-03,4719.02,January,West,"12,442,373",Monday,2022,0,2022-01,6.79
3,3,Mumbai,2022-01-04,4719.04,January,West,"12,442,373",Tuesday,2022,1,2022-01,6.79
4,4,Mumbai,2022-01-05,6895.06,January,West,"12,442,373",Wednesday,2022,2,2022-01,6.79
...,...,...,...,...,...,...,...,...,...,...,...,...
2185,2185,Pune,2022-12-27,4764.46,December,West,"3,124,458",Tuesday,2022,1,2022-12,5.79
2186,2186,Pune,2022-12-28,2787.45,December,West,"3,124,458",Wednesday,2022,2,2022-12,5.79
2187,2187,Pune,2022-12-29,3931.62,December,West,"3,124,458",Thursday,2022,3,2022-12,5.79
2188,2188,Pune,2022-12-30,2364.65,December,West,"3,124,458",Friday,2022,4,2022-12,5.79


In [147]:
df_miss= pd.read_csv(r'C:\Users\imran\Desktop\MyData\Python\Project\missing_val.csv')

x = df_miss['city'].isnull()
df_miss.loc[x]
df_miss['city'] = df_miss['city'].ffill()
df_miss.loc[x]
df_miss.loc[1]


# 1. Identify top 1 city by total kwh_consumed
# 2. Filter data for that city
# 3. Sort by date to ensure rolling works properly
# 4. Create 7-day rolling average
# 5. See result


city              Mumbai
date            1/2/2022
kwh_consumed     4834.08
month            2022-01
Name: 1, dtype: object

In [189]:
grouped = df_uses.groupby('city')['kwh_consumed'].agg('sum').reset_index()

grouped = grouped.sort_values(by = 'kwh_consumed', ascending = False).reset_index()
grouped

,index,city,kwh_consumed
0,2,Delhi,1848933.67
1,0,Bangalore,1827043.72
2,4,Mumbai,1820412.66
3,5,Pune,1818824.92
4,3,Kolkata,1800848.03
5,1,Chennai,1789114.52


In [201]:
top_city = df_uses.groupby('city')['kwh_consumed'].sum().idxmax()

#df_top_city = df_uses.loc[df_uses['city'] == top_city]

df_top_city

,Unnamed: 0,city,date,kwh_consumed,month_x,region,population,day,year,weekday,monyr,tariff_per_unit
365,365,Delhi,2022-01-01,4849.06,January,North,"11,034,555",Saturday,2022,5,2022-01,5.69
366,366,Delhi,2022-01-02,5066.87,January,North,"11,034,555",Sunday,2022,6,2022-01,5.69
367,367,Delhi,2022-01-03,6313.03,January,North,"11,034,555",Monday,2022,0,2022-01,5.69
368,368,Delhi,2022-01-04,4810.39,January,North,"11,034,555",Tuesday,2022,1,2022-01,5.69
369,369,Delhi,2022-01-05,0.00,January,North,"11,034,555",Wednesday,2022,2,2022-01,5.69
...,...,...,...,...,...,...,...,...,...,...,...,...
725,725,Delhi,2022-12-27,3303.54,December,North,"11,034,555",Tuesday,2022,1,2022-12,5.11
726,726,Delhi,2022-12-28,4117.36,December,North,"11,034,555",Wednesday,2022,2,2022-12,5.11
727,727,Delhi,2022-12-29,6483.31,December,North,"11,034,555",Thursday,2022,3,2022-12,5.11
728,728,Delhi,2022-12-30,3689.22,December,North,"11,034,555",Friday,2022,4,2022-12,5.11


In [241]:
#df_uses.loc[(df_uses['city'] == 'Mumbai').head(2) | (df_uses['city'] == 'Pune')].head(2)



#df_x= df_uses.loc[df_uses['city'].isin(['Mumbai','Pune', 'Delhi'])].groupby('city').head(2)

#df_x.groupby('city')['kwh_consumed'].agg('sum').idxmax()

df_x.groupby('city')['kwh_consumed'].agg('sum')

city
Delhi      9915.93
Mumbai    10430.14
Pune       9123.18
Name: kwh_consumed, dtype: float64

In [270]:
df_uses.groupby('city') ['kwh_consumed'].sum().sort_values(ascending = False).idxmax()

df_top_city = df_uses.loc[df_uses['city'] == 'Delhi']

df_top_city['x'] = df_top_city['kwh_consumed'].rolling(window = 7).mean().reset_index(drop = True)


df_top_city

C:\Users\imran\AppData\Local\Temp\ipykernel_8424\1806106855.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_city['x'] = df_top_city['kwh_consumed'].rolling(window = 7).mean().reset_index(drop = True)


,Unnamed: 0,city,date,kwh_consumed,month_x,region,population,day,year,weekday,monyr,tariff_per_unit,x
365,365,Delhi,2022-01-01,4849.06,January,North,"11,034,555",Saturday,2022,5,2022-01,5.69,NaN
366,366,Delhi,2022-01-02,5066.87,January,North,"11,034,555",Sunday,2022,6,2022-01,5.69,NaN
367,367,Delhi,2022-01-03,6313.03,January,North,"11,034,555",Monday,2022,0,2022-01,5.69,NaN
368,368,Delhi,2022-01-04,4810.39,January,North,"11,034,555",Tuesday,2022,1,2022-01,5.69,NaN
369,369,Delhi,2022-01-05,0.00,January,North,"11,034,555",Wednesday,2022,2,2022-01,5.69,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,725,Delhi,2022-12-27,3303.54,December,North,"11,034,555",Tuesday,2022,1,2022-12,5.11,NaN
726,726,Delhi,2022-12-28,4117.36,December,North,"11,034,555",Wednesday,2022,2,2022-12,5.11,NaN
727,727,Delhi,2022-12-29,6483.31,December,North,"11,034,555",Thursday,2022,3,2022-12,5.11,NaN
728,728,Delhi,2022-12-30,3689.22,December,North,"11,034,555",Friday,2022,4,2022-12,5.11,NaN
